In [1]:
%cd /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/CDHU-katrin_youtube_dalia/yolov5 

import torch
import utils
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import yaml
import os
import glob as glob
import requests
import cv2

%matplotlib inline

/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/CDHU-katrin_youtube_dalia/yolov5


In [2]:
# Training data path
data_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/datasets/Sverige/sverige_paths.yaml'

# Video frames path


In [3]:
def rename_df_cols(df):
    old_cols = list(df.columns)
    new_cols = ['epoch', 'train/box_loss', 'train/obj_loss',
       'train/cls_loss', 'metrics/precision', 'metrics/recall',
       'metrics/mAP_0.5', 'metrics/mAP_0.5:0.95', 'val/box_loss',
       'val/obj_loss', 'val/cls_loss', '/lr0', 'x/lr1', 'x/lr2']
    aux = dict(zip(old_cols, new_cols))
    df = df.rename(columns=aux)
    return df

def plot_losses(df_results):
    df_results.plot(x='epoch', y=['train/cls_loss', 'val/cls_loss'])
    plt.xlabel("Epoch")
    plt.ylabel("Cls_loss")
    #plt.show()
    

def modify_lr_hyperparam(hyperparam_file_path:str, lr_new:float):
    with open(hyperparam_file_path, 'r') as f:
        yaml_dict = yaml.safe_load(f)
        
        # Modify the 'lr' value in the sixth line
        yaml_dict['lr0'] = lr_new
        #print(yaml_dict['lr0'])
        
    # Write the modified YAML back to the file
    with open(hyperparam_file_path, 'w') as f:
        yaml.dump(yaml_dict, f)
        
        
"""degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.1  # image mixup (probability)
"""

def hyperparam_tuning(hyperparam_file_path:str, lr_new=0.1, degrees_new=0.0, translate_new=0.1, scale_new=0.9, shear_new=0.0, perspective_new=0.0, 
                      flipud_new=0.0, fliplr_new=0.5, mosaic_new=1.0, mixup_new=0.1):
    
    # Open hyperparamentes file 
    with open(hyperparam_file_path, 'r') as f:
        yaml_dict = yaml.safe_load(f)
        
        # Modify the 'lr' value in the sixth line
        yaml_dict['lr0'] = lr_new
        yaml_dict['degrees'] = degrees_new
        yaml_dict['translate']= translate_new
        yaml_dict['scale']= scale_new
        yaml_dict['shear']= shear_new
        yaml_dict['perspective']= perspective_new
        yaml_dict['flipud']= flipud_new
        yaml_dict['fliplr']= fliplr_new
        yaml_dict['mosaic']= mosaic_new
        yaml_dict['mixup']= mixup_new
        
        
    # Write the modified YAML back to the file
    with open(hyperparam_file_path, 'w') as f:
        yaml.dump(yaml_dict, f)
        

# Function to show validation predictions saved during training.
def show_valid_results(results_model_path, figure_size):
    validation_pred_images = glob.glob(f"{results_model_path}/*_pred.jpg")
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=figure_size)
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

#### Model

In [4]:
# Bringing best model from previous experiments
swe_model_name = 'lr0.01_b32_e100_f12_V5M6'
#results_V5M6 = rename_df_cols(pd.read_csv(f'Sverige/{model_name}/results.csv'))

In [21]:
def run_detection(model_name:str, test_data_path:str, test_name:str):
    model_path = f'Sverige/{model_name}/weights/best.pt'
    test_folder_path = f'{model_name}_TEST_{test_name}'
    
    !python detect.py \
    --weights {model_path} \
    --source {test_data_path} \
    --project Sverige \
    --name {test_folder_path} \
    --save-txt \
    --save-conf

In [10]:
def show_test_results(test_results_path, figure_size):
    test_images = glob.glob(f"{test_results_path}/*.jpeg")
    for pred_image in test_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=figure_size)
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

#### Test on video's frames

In [12]:
test_swe_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/'
run_detection(swe_model_name, test_swe_path, 'swe_mixed')

detect: weights=['Sverige/lr0.01_b32_e100_f12_V5M6/weights/best.pt'], source=/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Sverige, name=lr0.01_b32_e100_f12_V5M6_TEST_swe_mixed, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.10.8 torch-2.0.0+cu117 CUDA:0 (Quadro RTX 5000, 16125MiB)

Fusing layers... 
Model summary: 276 layers, 35254692 parameters, 0 gradients, 48.9 GFLOPs
image 1/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-2VUFLVeCd4-137_0.jpeg: 384x640 1 s_soc, 74.4ms
image 2/6965 /zpool/beast-mirror/labour-movements-m

image 46/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-Vn_tfRebOU-135_6.jpeg: 384x640 (no detections), 18.4ms
image 47/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-Vn_tfRebOU-135_7.jpeg: 384x640 (no detections), 12.3ms
image 48/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-Vn_tfRebOU-135_8.jpeg: 384x640 1 s_soc, 18.3ms
image 49/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-Vn_tfRebOU-135_9.jpeg: 384x640 (no detections), 17.7ms
image 50/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-W90HuZXR8g-299_0.jpeg: 384x640 1 s_soc, 12.1ms
image 51/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/-W90HuZXR8g-299_1.jpeg: 384x640 1 s_soc, 11.7ms
image 52/6965 /z

image 96/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_1.jpeg: 384x640 (no detections), 12.6ms
image 97/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_2.jpeg: 384x640 (no detections), 12.7ms
image 98/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_3.jpeg: 384x640 1 s_soc, 9.7ms
image 99/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_4.jpeg: 384x640 1 s_soc, 10.2ms
image 100/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_5.jpeg: 384x640 2 s_socs, 12.2ms
image 101/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/01RJhEMnk0E-137_6.jpeg: 384x640 1 s_soc, 12.8ms
image 102/6965 /zpool/

image 145/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0Nz4zjpFeWY-136_7.jpeg: 384x640 (no detections), 15.4ms
image 146/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0Nz4zjpFeWY-136_8.jpeg: 384x640 (no detections), 15.8ms
image 147/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0Nz4zjpFeWY-136_9.jpeg: 384x640 (no detections), 14.0ms
image 148/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0OJ5Zlyc-_M-136_0.jpeg: 384x640 (no detections), 15.1ms
image 149/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0OJ5Zlyc-_M-136_1.jpeg: 384x640 (no detections), 15.5ms
image 150/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0OJ5Zlyc-_M-136_2.jpeg: 384x640 (no detect

image 194/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_0.jpeg: 384x640 (no detections), 18.5ms
image 195/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_1.jpeg: 384x640 1 s_svd, 13.3ms
image 196/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_2.jpeg: 384x640 2 s_socs, 18.2ms
image 197/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_3.jpeg: 384x640 (no detections), 17.6ms
image 198/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_4.jpeg: 384x640 (no detections), 18.1ms
image 199/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/0e1FwJKrBzY-135_5.jpeg: 384x640 (no detections), 11.5ms
i

...

image 6725/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_3.jpeg: 384x640 1 s_soc, 12.2ms
image 6726/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_4.jpeg: 384x640 (no detections), 12.2ms
image 6727/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_5.jpeg: 384x640 (no detections), 9.5ms
image 6728/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_6.jpeg: 384x640 (no detections), 12.2ms
image 6729/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_7.jpeg: 384x640 (no detections), 12.6ms
image 6730/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/y92cKvwPJ78-137_8.jpeg: 384x640 (no detection

image 6774/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yMxGGGMbSb8-133_2.jpeg: 384x640 (no detections), 15.1ms
image 6775/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yMxGGGMbSb8-133_3.jpeg: 384x640 (no detections), 18.9ms
image 6776/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yQC2-Z6-w3s-136_0.jpeg: 384x640 (no detections), 15.4ms
image 6777/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yQC2-Z6-w3s-136_1.jpeg: 384x640 (no detections), 16.0ms
image 6778/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yQC2-Z6-w3s-136_2.jpeg: 384x640 (no detections), 15.2ms
image 6779/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yQC2-Z6-w3s-136_3.jpeg: 384x640 (no 

image 6823/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yg55O3TFnJc-137_6.jpeg: 384x640 (no detections), 11.8ms
image 6824/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yg55O3TFnJc-137_7.jpeg: 384x640 1 s_soc, 11.9ms
image 6825/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yg55O3TFnJc-137_8.jpeg: 384x640 (no detections), 12.4ms
image 6826/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yg55O3TFnJc-137_9.jpeg: 384x640 (no detections), 9.7ms
image 6827/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yhBE3Rm2IvA-137_0.jpeg: 384x640 (no detections), 11.4ms
image 6828/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/yhBE3Rm2IvA-137_1.jpeg: 384x640 (no detection

image 6872/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zPJDL1AyjEk-137_6.jpeg: 384x640 (no detections), 9.8ms
image 6873/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zPJDL1AyjEk-137_7.jpeg: 384x640 (no detections), 9.7ms
image 6874/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zPJDL1AyjEk-137_8.jpeg: 384x640 (no detections), 11.5ms
image 6875/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zPJDL1AyjEk-137_9.jpeg: 384x640 (no detections), 9.6ms
image 6876/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zSUHxZmCKPw-137_0.jpeg: 384x640 (no detections), 12.2ms
image 6877/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zSUHxZmCKPw-137_1.jpeg: 384x640 1 s_svd

image 6922/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zkgniTXyt4c-133_6.jpeg: 384x640 (no detections), 14.0ms
image 6923/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zkgniTXyt4c-133_7.jpeg: 384x640 (no detections), 11.5ms
image 6924/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zkgniTXyt4c-133_8.jpeg: 384x640 (no detections), 10.0ms
image 6925/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zkgniTXyt4c-133_9.jpeg: 384x640 (no detections), 9.5ms
image 6926/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zr2OJ8XTRxE-137_0.jpeg: 384x640 (no detections), 12.3ms
image 6927/6965 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden_mixed/zr2OJ8XTRxE-137_1.jpeg: 384x640 (no d

In [22]:
test_swe_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden/DIKforbundet/AAHMZ3uhgQg/'
run_detection(swe_model_name, test_swe_path, 'AAHMZ3uhgQg')

detect: weights=['Sverige/lr0.01_b32_e100_f12_V5M6/weights/best.pt'], source=/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden/DIKforbundet/AAHMZ3uhgQg/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Sverige, name=lr0.01_b32_e100_f12_V5M6_TEST_AAHMZ3uhgQg, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.10.8 torch-2.0.0+cu117 CUDA:0 (Quadro RTX 5000, 16125MiB)

Fusing layers... 
Model summary: 276 layers, 35254692 parameters, 0 gradients, 48.9 GFLOPs
image 1/10 /zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/swedish_test_data/sweden/DIKforbundet/AAHMZ3uhgQg/AAHMZ3uhgQg-134_0.jpeg: 384x640 (no detections), 44.6ms
image 2